# Opentrons OT-2 protocol quick-start

This notebook is a tool for new Opentrons users developing their protocols using the Python API. It includes several custom functions that assist with accurate pipetting.

## Header info

If you make any significant modificiation to this file, please update the infomation in this cell...

### Changelog
**Version:** 1<br>
_Comitted by:_ Alaric Taylor <alaric.taylor@ucl.ac.uk><br>
_Commit date:_ 2020-09-24<br>
* First committ

**Version:** 2<br>
_Comitted by:_ Yann Mamie <y.mamie@ucl.ac.uk>:<br>
_Commit date:_ 2020-09-25<br>
* Added a wetting, 2 mix and 2 transfer functions

**Version:** 3<br>
_Comitted by:_  Alaric Taylor <alaric.taylor@ucl.ac.uk>:<br>
_Commit date:_ 2020-09-25<br>
* Restructured Header info
* Added reference links
* Removed additional aspiration command from `custom_wetting()` function
* Removed pick_up_tip() and return_tip() from `custom_mixing_basic()` function (not flexible enough for use in protocols)
* ...then commented-out `custom_mixing_basic()` and replaced it with `custom_mixing_bottom_to_top()`, which was much more efficient.
* Significant overhaul of `custom_mixing_static()`...now mixing in the middle of the liquid
* Changes to `custom_transfer_simple()`:
    * Now called `custom_transfer_forward()`
    * Don't need `pipette_max_V` argument as this is an attribute of the `pipette` instance
* Re-build of `custom_transfer_reverse()`
* Added `rate` variable to `custom_aspirate()` function
* Added `custom_touch_tip()` which swirls an arc around the opening of a circular tube to remove droplets.

## Library imports

In [17]:
from opentrons import protocol_api

In [3]:
# Used within custom functions
import numpy as np
from scipy.interpolate import interp1d
import math
import time

## Metadata
Specifying `apiLevel` in the metadata is important... see [protocol API version control](https://docs.opentrons.com/v2/versioning.html) for more info.

In [19]:
metadata = {
    'apiLevel': '2.6', # maximum supported API level is visible in the Opentrons App
    'protocolName':'My Protocol',
    'description':'Simple protocol to get started using the OT-2',
    'author': 'Alaric Taylor'}

## References and tools
* Opentrons website
    * [API documention](https://docs.opentrons.com/)
    * [PDF reference guide](https://docs.opentrons.com/OpentronsPythonAPIV2.pdf)
    * [Support articles](https://support.opentrons.com/)
    * [Protocol library](https://protocols.opentrons.com/)
* GitHub
    * [Library page](https://libraries.io/github/Opentrons/opentrons) (high-level)
    * [API](https://github.com/Opentrons/opentrons/tree/edge/api)
    * [API v2 source code](https://github.com/Opentrons/opentrons/tree/edge/api/docs/v2) (more detailed)
* PyPi (package distribution)
    * [PyPi](https://pypi.org/project/opentrons/)


### Labware
* Standard Opentrons labware [library](https://labware.opentrons.com/)
* [Labware creator](https://labware.opentrons.com/create/) (for custom labware defenitions)


### Markdown
 * Basic [syntax](https://www.markdownguide.org/basic-syntax/)
 * Wordpress [quick reference](https://wordpress.com/support/markdown-quick-reference/)
 * [Cheat sheet](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#code)

# Custom pipetting functions

These functions give fine control over the height at which an aspiration is made. This is implemented by tracking meniscus heights in each well. Thereby,
* maximising the accuracy of the pipette stroke (by pipetting at the top of a liquid)
* avoiding the formation of droplets along the outer pipette shaft
* preventing silent failures of a protocol e.g. if stock solution volumes are low and a pipette stroke fails to pick up liquid

In [20]:
def custom_aspirate(pipette, volume, location, tip_submersion_depth = 2, safety_height = 0.5, rate=1.0):
    # aspiration_depth (in mm) beneath meniscus
    # safety_height prevents the tip from crashing into bottom of the well
    
    # Check a headroom attribute has been defined for the current location 
    if not hasattr(location, "headroom"): # If not, break 
        print('Headroom not set for ' + str(location))
        
    # Check a gradations attribute has been defined for the current location 
    if not hasattr(location, "volume_headroom_functions"): # If not, break 
        print('volume_headroom_functions (based upon gradations) not set for ' + str(location))
        
    # Unpack the vh functions that have been attributed to this tube
    v_given_h = location.volume_headroom_functions[0]
    h_given_v = location.volume_headroom_functions[1]
    
    
    final_volume = v_given_h(location.headroom) - volume
    # print('final_volume ', str(final_volume))
    depth_to_aspirate = h_given_v(final_volume) + tip_submersion_depth
    # print('depth_to_aspirate ', str(depth_to_aspirate))
    
    location.headroom = h_given_v(final_volume)
    
    # Safety checking
    # ...avoidance of these errors are also covered by the use 
    # of specified extrapolation fill_values in the gradations_to_vh() function
    if depth_to_aspirate > location._depth - safety_height:
        depth_to_aspirate = location._depth - safety_height  # e.g. 0.5mm above the bottom of the well
        print('Warning, aspiration height very low...')
    
    if depth_to_aspirate < 0: 
        depth_to_aspirate = 0 # i.e. at the top of the well
    
    pipette.move_to(location.top(-depth_to_aspirate))
    pipette.aspirate(volume, rate=rate)
    


The `custom_dispense()` function record the volume of fluid added to a well.

In [21]:
def custom_dispense(pipette, volume, location):
    # Air dispensing
    
    # Check a headroom attribute has been defined for the current location 
    if not hasattr(location, "headroom"): # If not, break 
        print('Headroom not set for ' + str(location))
        
    # Check a gradations attribute has been defined for the current location 
    if not hasattr(location, "volume_headroom_functions"): # If not, break 
        print('volume_headroom_functions (based upon gradations) not set for ' + str(location))
        
    # Unpack the vh functions that have been attributed to this tube
    v_given_h = location.volume_headroom_functions[0]
    h_given_v = location.volume_headroom_functions[1]
    
    current_filled_volume = v_given_h(location.headroom)
    
    # Set new headroom from volume info
    location.headroom = h_given_v(current_filled_volume + volume)
    
    pipette.move_to(location.top())
    pipette.dispense(volume)

In [22]:
def custom_wetting(volume, location, pipette):
    custom_aspirate(pipette, volume, location)
    custom_dispense(pipette, volume, location)
    pipette.blow_out()
    # pipette.touch_tip(-2)
    custom_touch_tip(location, pipette)

Alaric: I think we should drop `custom_mixing_basic()` in favour of `custom_mixing_bottom_to_top()`...

In [23]:
#def custom_mixing_basic(cycles, volume, location, pipette): # Using basic function, height changed for dispensing
#    for i in range(0, cycles):
#        custom_aspirate(pipette, volume, location) # This is going to take liquid from the top of the meniscus each time...
#        custom_dispense(pipette, volume, location) # This is going to move to the very top of the well each time...
#    pipette.move_to(location.top())
#    pipette.blow_out()
#    pipette.touch_tip()

The `custom_mixing_bottom_to_top()` takes liquid from the bottom of the well and pipettes it to the top of the well i.e. layering the solution in order to aid mixing.

In [ ]:
def custom_mixing_bottom_to_top(cycles, volume, location, pipette): # Using basic function, height changed for dispensing
    
    aspirate_depth = location._depth -1 # 1mm above the bottom of the well
    dispense_depth = location.headroom + 2 # 2mm above the mensicus level 

    for i in range(0, cycles):
        pipette.move_to(location.top(aspirate_depth))
        pipette.aspirate(volume)
        
        pipette.move_to(location.top(dispense_depth))
        pipette.dispense(volume)
        
        # the .aspirate() and .dispense() functions used above 
        # do not track changes in headroom...because they balance each other.
    
    pipette.move_to(location.top())
    pipette.blow_out()
    custom_touch_tip(location, pipette)

In [24]:
def custom_mixing_static(cycles, volume, location, tip, pipette): # Using mix function, static
    
    # Check a headroom attribute has been defined for the current location 
    if not hasattr(location, "headroom"): # If not, break 
        print('Headroom not set for ' + str(location))
        
    # Check a gradations attribute has been defined for the current location 
    if not hasattr(location, "volume_headroom_functions"): # If not, break 
        print('volume_headroom_functions (based upon gradations) not set for ' + str(location))
        
    # Unpack the vh functions that have been attributed to this tube
    v_given_h = location.volume_headroom_functions[0]
    h_given_v = location.volume_headroom_functions[1]
    
    maximum_volume = v_given_h(location.headroom)
    minimum_volume = v_given_h(location.headroom) - volume
    
    if minimum_volume <= 0:
        print('Warning, not enough liquid in well to mix properly')
    
    maximum_headheight = h_given_v(minimum_volume) # The lowest point the meniscus will reach during aspiration    
    safety_depth = location._depth -1 # i.e. within 1mm of the well bottom
    
    if maximum_headheight > safety_depth:
        print('Warning, not enough liquid in well to mix properly')
        maximum_headheight = safety_depth
    
    # Mix half-way between the bottom of the well and 
    mixing_depth = 0.5*(maximum_headheight + location._depth)
    pipette.move_to(location.top(-mixing_depth))
    
    for i in range(0, cycles):
        pipette.aspirate(volume)
        pipette.dispense(volume)
        
        # the .aspirate() and .dispense() functions used above 
        # do not track changes in headroom...because they balance each other.

    pipette.move_to(location.top())
    pipette.blow_out()
    # pipette.touch_tip(-2)
    custom_touch_tip(location, pipette)

In [25]:
def custom_transfer_forward(volume, source, destination, pipette, air_gap=2, pre_wet=True):
    # for low-viscosity/aqueous solutions, such as buffers, diluted acids or alkalis
    
    if volume <= pipette.max_volume:
        if volume + air_gap > pipette.max_volume:
            print('Warning, in order to include an air_gap this transfer will be split into several passes...')
    
    if volume + air_gap <= pipette.max_volume:
        volume_list = [volume]
    else:
        number_of_passes = np.ceil((volume / (pipette.max_volume - air_gap)))
        volume_per_pass = volume/number_of_passes
        volume_list = np.ones(number_of_passes)*volume_per_pass
        
    if pre_wet == True:
        custom_wetting(volume_list[0]+air_gap, source, pipette)
        
    for pass_volume in volume_list:
        custom_aspirate(pipette, pass_volume, source)
        custom_touch_tip(source, pipette)
        
        if air_gap > 0:
            pipette.move_to(source.top())
            pipette.aspirate(air_gap, source.top())
        
        custom_dispense(pipette, pass_volume + air_gap, destination)
        pipette.blow_out()
        
        # deciding NOT to touch_tip() on the destination
        # as this could cause backwards contamination
    
    # Remove droplets from tip after full sequence is complete
    # pipette.touch_tip(source.top(-2))
    custom_touch_tip(source, pipette)
    

In [26]:
def custom_transfer_reverse(volume, source, destination, pipette, disposal_volume=5, rate=0.5, pre_wet=False):
    # for pipetting solutions with a high viscosity or a tendency to foam
    # ...also recommended for dispensing small volumes of low-viscosity solutions.
    
    if volume <= pipette.max_volume:
        if volume + disposal_volume > pipette.max_volume:
            print('Warning, in order to include a disposal_volume this transfer will be split into several passes...')
    
    if volume + disposal_volume <= pipette.max_volume:
        volume_list = [volume]
    else:
        number_of_passes = np.ceil((volume / (pipette.max_volume - disposal_volume)))
        volume_per_pass = volume/number_of_passes
        volume_list = np.ones(number_of_passes)*volume_per_pass
        
    if pre_wet == True:
        custom_wetting(volume_list[0]+disposal_volume, source, pipette)
        
    for pass_volume in volume_list:
        custom_aspirate(pipette, pass_volume + disposal_volume, source, rate=rate)
        
        #pipette.touch_tip(-2)
        custom_touch_tip(source, pipette)
        
        custom_dispense(pipette, pass_volume, destination)
        
        # Return the remainder to the source 
        custom_dispense(pipette, disposal_volume, source)
        pipette.blow_out()
    
    # Remove droplets from tip after full sequence is complete
    # pipette.touch_tip(source.top(-2))
    custom_touch_tip(source, pipette)


The `custom_touch_tip()` function removes droplets from the shaft of a pipette tip by running the tip around the perimiter of a circular tube.

In [ ]:
def custom_touch_tip(well, pipette, depth=-2, rotations=1, increments=50):
    
    if location._shape == 'circular':
        radius = well._diameter / 2
        
        theta = 0.0 # degrees
        theta_step = (360 * rotations) / increments
        
        pipette.move_to(well.top(depth))
        
        for i in range(increments):
            # move straight there
            pipette.move_to(well.top(z=depth, radius=radius, degrees=theta), strategy='direct')

            theta += theta_step
        
        pipette.move_to(well.top(depth), strategy='direct')
        
    else:
        print('Well is not circular, so just using standard touch_tip() command')
        pipette.touch_tip(location=well, v_offset=depth)

## Attributes for meniscus height tracking

In [27]:
####################################
# Generate volume_headroom_functions
# for each type of tube in use from 
# measured gradations...

def gradations_to_vh(gradations):
    # Converts gradations (numpy array):
    # to tuple containg the v_given_h and h_given_v interpolation functions
    
    # Slicing the gradation array
    h = gradations[:,0] # headroom i.e. meniscus to top of container
    v = gradations[:,1] # fill volume
    
    maximum_headroom = gradations[:,0][-1]
    maximum_volume = gradations[:,1][0]
    
    # Interpolation functions
    # if kind='cubic', there is a risk the interpolated function
    # becomes negative close to the bottom of the well
    v_given_h = interp1d(h, v, kind='linear', assume_sorted=False, bounds_error=False, fill_value=(maximum_volume,0))
    h_given_v = interp1d(v, h, kind='linear', assume_sorted=False, bounds_error=False, fill_value=(maximum_headroom,0))
    
    # The order of the returned function is referenced by 
    # the custom_aspirate() and custom_dispense() functions
    return [v_given_h, h_given_v] 

Below, we define the gradation profiles for our standard tubes/wells. These are then converted to 'volume_headroom_functions' using `gradations_to_vh()`, above. The 'volume_headroom_functions' are then ascribed to each well instance at the beginning of the protocol.

In [28]:
#####################################
# Gradations defined as a numpy array 
# (cast from a list of sublists)
# Each sublost is structured as [meniscus_to_top_in_mm, uL_volume]

#####
# 1.5ml Eppendorph tubes
gradations = np.array([[1,1500], # Top gradation of container i.e. maximum liquid fill
                       [5,1000],
                       [10,500],
                       [13, 20],
                       [15,0]]) # Bottom of container i.e. no liquid

tube_vh = gradations_to_vh(gradations)

#####
# 15ml Falcon tubes


#####
# 50ml Falcon tubes


#####
# etc.



### OT-2 deck layout

|	|	|	|
|:--:	|:--:	|:-----:	|
| 10 	| 11 	| Trash 	|
|  7 	|  8 	|   9   	|
|  4 	|  5 	|   6   	|
|  1 	|  2 	|   3   	|

### Well referencing schema

| <blank>| <blank>| <blank>| <blank>| <blank>|
|:--:	|:--:	|:-----:	|:-----:	|:-----:	|
|<blank>	|**1**	|**2**	| **3** | … |
| **A**| <blank>| <blank>| <blank>| <blank>|
| **B** | <blank>| <blank>| <blank>| <blank>|
|  **C** | <blank>| <blank>| <blank>| <blank>|
|  ⋮  	| <blank>| <blank>| <blank>| <blank>|

# The Protocol

In [29]:
# protocol run function. the part after the colon lets your editor know
# where to look for autocomplete suggestions...but doesn't seem to work with Jupyter

def run(protocol: protocol_api.ProtocolContext):
    
    #############
    # Deck layout
    
    # Labware
    tiprack = protocol.load_labware('opentrons_96_tiprack_300ul',
                                        location='1',
                                        label='tiprack')
    tubes = protocol.load_labware('opentrons_24_aluminumblock_nest_1.5ml_screwcap',
                                        location='2',
                                        label='tubes')
    
    # Pipettes
    p50 = protocol.load_instrument('p50_single', 'left', tip_racks=[tiprack])
    p300 = protocol.load_instrument('p300_single', 'right', tip_racks=[tiprack])
    
    
    ######################
    # Named tubes and tips
    
    # Tips
    methanol_tip = tiprack['A1']

    # Tubes
    methanol = tubes['A1']
    water = tubes['A2']
    mixture = tubes['A3']
    
    # Setting instance attributes for 
    # all wells... simpler than 
    # extending classes and labware defenitions?
    #
    # Add for loop for each labware item
    for well in tubes.wells():
        well.headroom = well._depth # Initiated, but considered empty
        
        # Generate volume_headroom_functions uniques for
        # each well type by defining gradations and then 
        # running this through the gradations_to_vh() function, above
        well.volume_headroom_functions = tube_vh
    
    # Modify instance attributes for vials that start the protocol
    # filled with liquid...
    # Headroom measured in mm from meniscus to top of tube
    methanol.headroom = 5
    

    ####################
    ## General debugging
    # protocol.comment('Test comment')
    # protocol.pause('Test pause')
    
    ## Debugging instance attributes
    #attrs = vars(methanol)
    #print(str(attrs))
    
    #########
    # Actions
    
    p50.pick_up_tip(methanol_tip)
    p50.move_to(methanol.bottom(2)) # move to 2mm above the bottom of well A1
    custom_aspirate(p50, 50, methanol)
    #p50.transfer(50,methanol,water)
    p50.return_tip()
    p50.transfer(50,methanol,water)
    
    #p50.drop_tip() # drop in trash
    
    protocol.pause()
    time.sleep(1)
    protocol.resume()

## Simulating protocol
1. Save as Python file (File > Download as...)
2. Inside python terminal, run:

`opentrons_simulate my_protocol.py`

Note, if using custom labware defenitions, append the following:

`--custom-labware-path="C:\Custom Labware"`
